In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '${:,.2f}'.format

In [11]:
csv_path = 'Resources/purchase_data.csv'
data_df = pd.read_csv(csv_path)

list(data_df)

['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price']

In [3]:
# Number of Players

num_players = data_df["SN"].nunique()

players_df = pd.DataFrame({"Number of Players": [num_players]})
players_df

,Number of Players
0,576


In [4]:
# Purchasing Analysis (Total)

purchasing_data = {
    "Number of Unique Items": [data_df["Item Name"].nunique()],
    "Average Price": [round(data_df["Price"].mean(), 2)],
    "Number of Purchases": [data_df["Purchase ID"].count()],
    "Total Revenue": [data_df["Price"].sum()]
}

purchasing_df = pd.DataFrame(purchasing_data)
purchasing_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
# Gender Demographics

gender_gpby = data_df.groupby("Gender")

gender_tally_df = pd.DataFrame(gender_gpby["SN"].nunique())
gender_tally_df = gender_tally_df.rename(columns={"SN": "Total Count"})
gender_tally_df["Percentage of Players"] = (100 * gender_tally_df["Total Count"] / num_players).map("{:.2f}%".format)
gender_tally_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [6]:
# Purchasing Analysis (Gender)

gender_analysis_df = pd.DataFrame()

gender_analysis_df["Purchase Count"] = gender_gpby["Purchase ID"].count()
gender_analysis_df["Average Purchase Price"] = gender_gpby["Price"].mean()
gender_analysis_df["Total Purchase Value"] = gender_gpby["Price"].sum()
gender_analysis_df["Avg Total Purchase per Person"] = gender_analysis_df["Total Purchase Value"] / gender_tally_df["Total Count"]

gender_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
# Age Demographics

age_bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 1000]
bin_names = ["< 10", "10 - 14", "15 - 19", "20 - 24", "25 - 29", "30 - 34", "35 - 39", "> 40"]

data_df["Age Range"] = pd.cut(data_df["Age"], age_bins, labels=bin_names, include_lowest=True)

age_gpby = data_df.groupby("Age Range")

age_tally_df = pd.DataFrame(age_gpby["SN"].nunique())
age_tally_df = age_tally_df.rename(columns={"SN": "Total Count"})
age_tally_df["Percentage of Players"] = (100 * age_tally_df["Total Count"] / num_players).map("{:.2f}%".format)
age_tally_df

,Total Count,Percentage of Players
Age Range,,
< 10,17,2.95%
10 - 14,22,3.82%
15 - 19,107,18.58%
20 - 24,258,44.79%
25 - 29,77,13.37%
30 - 34,52,9.03%
35 - 39,31,5.38%
> 40,12,2.08%


In [8]:
# Purchasing Analysis (Age)

age_analysis_df = pd.DataFrame()

age_analysis_df["Purchase Count"] = age_gpby["Purchase ID"].count()
age_analysis_df["Average Purchase Price"] = age_gpby["Price"].mean()
age_analysis_df["Total Purchase Value"] = age_gpby["Price"].sum()
age_analysis_df["Avg Total Purchase per Person"] = age_analysis_df["Total Purchase Value"] / age_tally_df["Total Count"]

age_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
< 10,23,$3.35,$77.13,$4.54
10 - 14,28,$2.96,$82.78,$3.76
15 - 19,136,$3.04,$412.89,$3.86
20 - 24,365,$3.05,"$1,114.06",$4.32
25 - 29,101,$2.90,$293.00,$3.81
30 - 34,73,$2.93,$214.00,$4.12
35 - 39,41,$3.60,$147.67,$4.76
> 40,13,$2.94,$38.24,$3.19


In [19]:
# Top Spenders

players_gpby = data_df.groupby("SN")

# get the 5 highest spenders & their data, then sort & get top 5 using iloc
top_spenders_df = players_gpby["Price"].sum().reset_index()
top_spenders_df["Purchase Count"] = players_gpby["Purchase ID"].count().reset_index()["Purchase ID"]
top_spenders_df["Average Purchase Price"] = top_spenders_df["Price"] / top_spenders_df["Purchase Count"]
top_spenders_df = top_spenders_df.sort_values("Price", ascending=False).iloc[:5]

# do housekeeping
top_spenders_df = top_spenders_df.set_index("SN")
top_spenders_df = top_spenders_df.rename(columns={"Price": "Total Purchase Value"})
top_spenders_df = top_spenders_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

top_spenders_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [36]:
# Most Popular Items

most_pop_gpby = data_df[["Item ID", "Item Name", "Price", "Purchase ID"]].groupby(["Item ID", "Item Name"])

# get the 5 most frequently purchased items & their data
by_item_df = most_pop_gpby["Purchase ID"].count().reset_index()
by_item_df["Item Price"] = most_pop_gpby["Price"].mean().reset_index()["Price"]
by_item_df["Total Purchase Value"] = most_pop_gpby["Price"].sum().reset_index()["Price"]

# do housekeeping
by_item_df = by_item_df.rename(columns={"Purchase ID": "Purchase Count"})
by_item_df = by_item_df.set_index("Item ID")

# sort & get top 5 using iloc
most_pop_df = by_item_df.sort_values("Purchase Count", ascending=False).iloc[:5]

most_pop_df

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [37]:
# Most Profitable Items

most_profitable_df = by_item_df.sort_values("Total Purchase Value", ascending=False).iloc[:5]

most_profitable_df

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
